<a href="https://colab.research.google.com/github/MatiasMercandino/predicciones-ventas-/blob/main/sales_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explora los datos: ¿Qué necesitas para limpiar estos datos?

# ¿Hay valores faltantes en el conjunto de datos? Es posible que algunas tiendas no informen de todos los datos a causa de problemas técnicos u otros problemas. De ser así, trata con ello apropiadamente.

# -Información de la data

In [ ]:
import pandas as pd
import numpy as np
filename = "/content/sales_predictions.csv"
df = pd.read_csv(filename)
df.info()
#Tienen datos nulos Item_Weight y Outlet_Size                

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


# -Datos estadisticos del data

In [ ]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


# -Ordenar por año de apertura Tienda


In [ ]:
#en 1985 hay puros datos NaN en la columna de PESO_Producto
#Hice esto ya que en los años mas antiguos me imagino que habran mayor errores de traspaso de datos
df=df.sort_values("Outlet_Establishment_Year",ascending=True)
df.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
4338,FDH21,NaN,Low Fat,0.031074,Seafood,157.5604,OUT027,1985,Medium,Tier 3,Supermarket Type3,5704.5744
3689,FDD53,NaN,Low Fat,0.044008,Frozen Foods,43.7454,OUT027,1985,Medium,Tier 3,Supermarket Type3,671.1264
1139,FDH26,NaN,Regular,0.034532,Canned,142.2496,OUT027,1985,Medium,Tier 3,Supermarket Type3,2399.5432
1138,FDM21,NaN,Low Fat,0.064052,Snack Foods,256.1646,OUT027,1985,Medium,Tier 3,Supermarket Type3,7472.2734
3691,FDY36,NaN,Low Fat,0.016477,Baking Goods,74.3380,OUT019,1985,Small,Tier 1,Grocery Store,146.4760


# -Contar valores

1985 ==> 1987 ==> 1997==> 1998 ==> 1999 ==> 2002 ==>  2004 ==>  2007 ==>  2009

In [ ]:
df["Outlet_Establishment_Year"].value_counts()

1985    1463
1987     932
1997     930
1999     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: Outlet_Establishment_Year, dtype: int64



1.   Pondria al azar cualquiera de los 3 en los datos que tienen nan
2.   Existen muchos nan, y hay pocos Size=High. Quizas podria reemplazarlas en las nan







In [ ]:
#Para predecir el modelo, debere tranformarlas a dummy
df["Outlet_Size"].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

# Agrupar por año la suma de sus pesos


1. Se puede ver claramente que los datos nulos estan todos en el año 1985  
2. Se podria sacar un promedio entre los otros años y agregarlo al año faltante, interpolar u algun otro metodo.



In [ ]:
df.groupby(df["Outlet_Establishment_Year"])[["Item_Weight"]].sum()

,Item_Weight
Outlet_Establishment_Year,
1985,0.000
1987,12121.730
1997,11966.125
1998,7166.800
1999,12013.225
2002,11751.840
2004,11931.295
2007,11877.495
2009,11946.465


1. Se puede ver que en los años 1998, 2002 y 2007 no existen datos, por lo que en estos años podrian haber NaN. 
2. Podria ser que en aquellas filas donde sean tipo Tier 2, los datos nulos los reemplazo con high
2. Se podria crear variables dummy para saber cuantos hay de cada Size
3. Existen pocos Size=High por lo que podria rellenar los datos faltantes igualandolos a high
4. Primera conclusion, a todos los datos nulos correspondientes al año 1985 les pondremos high en la columna Size

In [ ]:
df.groupby(df["Outlet_Establishment_Year"])[["Outlet_Size"]].count()

,Outlet_Size
Outlet_Establishment_Year,
1985,1463
1987,932
1997,930
1998,0
1999,930
2002,0
2004,930
2007,0
2009,928


In [ ]:
df.groupby(df["Outlet_Size"])[["Item_Weight"]].count()

,Item_Weight
Outlet_Size,
High,932
Medium,1858
Small,1860


In [ ]:
df2=pd.get_dummies(data=df, columns = ["Outlet_Size"],drop_first=False)
df2

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small
4338,FDH21,NaN,Low Fat,0.031074,Seafood,157.5604,OUT027,1985,Tier 3,Supermarket Type3,5704.5744,0,1,0
3689,FDD53,NaN,Low Fat,0.044008,Frozen Foods,43.7454,OUT027,1985,Tier 3,Supermarket Type3,671.1264,0,1,0
1139,FDH26,NaN,Regular,0.034532,Canned,142.2496,OUT027,1985,Tier 3,Supermarket Type3,2399.5432,0,1,0
1138,FDM21,NaN,Low Fat,0.064052,Snack Foods,256.1646,OUT027,1985,Tier 3,Supermarket Type3,7472.2734,0,1,0
3691,FDY36,NaN,Low Fat,0.016477,Baking Goods,74.3380,OUT019,1985,Tier 1,Grocery Store,146.4760,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3600,FDX31,20.35,Regular,0.014886,Fruits and Vegetables,234.4958,OUT018,2009,Tier 3,Supermarket Type2,3271.7412,0,1,0
5053,NCO53,16.20,Low Fat,0.175898,Health and Hygiene,183.7608,OUT018,2009,Tier 3,Supermarket Type2,2940.1728,0,1,0
7710,NCR18,15.85,Low Fat,0.020571,Household,41.6112,OUT018,2009,Tier 3,Supermarket Type2,255.6672,0,1,0
5750,FDV26,20.25,Regular,0.076471,Dairy,196.1794,OUT018,2009,Tier 3,Supermarket Type2,2731.1116,0,1,0


# -Transformando data colocando valores a los NaN

In [ ]:
for i in df["Outlet_Establishment_Year"]:
  if i == 1985:
    df["Item_Weight"]=round(df["Item_Weight"].mean(),2)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
4338,FDH21,12.86,Low Fat,0.031074,Seafood,157.5604,OUT027,1985,Medium,Tier 3,Supermarket Type3,5704.5744
3689,FDD53,12.86,Low Fat,0.044008,Frozen Foods,43.7454,OUT027,1985,Medium,Tier 3,Supermarket Type3,671.1264
1139,FDH26,12.86,Regular,0.034532,Canned,142.2496,OUT027,1985,Medium,Tier 3,Supermarket Type3,2399.5432
1138,FDM21,12.86,Low Fat,0.064052,Snack Foods,256.1646,OUT027,1985,Medium,Tier 3,Supermarket Type3,7472.2734
3691,FDY36,12.86,Low Fat,0.016477,Baking Goods,74.3380,OUT019,1985,Small,Tier 1,Grocery Store,146.4760


# -Codigos para saber los datos nulos
